# **거래까지 걸린 시간 찾기**

In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from tqdm import tqdm

In [2]:
df = pd.read_csv("C:/Users/whileduck/Desktop/빈좌석/클래식_빈좌석포함데이터.csv", low_memory = False)

display(df.head())
print(df.shape)

,seat,층,블록,열,넘버,X,Y,Z,대칭점,무대_밑_좌우시야각,...,공연월,공연일,공연연월,전체공연시간,전체거래시간,누적검색량,거래까지걸린시간,거래까지걸린시간_시간,거래까지걸린시간_등수,좌석 오픈 여부
0,1층 A블록1열 1,1층,A블록,1,1,1451,542,-93,1층 E블록1열 9,21,...,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1,32.0,1
1,1층 A블록1열 2,1층,A블록,1,2,1406,555,-93,1층 E블록1열 8,22,...,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1,32.0,1
2,1층 A블록1열 3,1층,A블록,1,3,1361,568,-93,1층 E블록1열 7,23,...,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1,32.0,1
3,1층 A블록1열 4,1층,A블록,1,4,1315,580,-93,1층 E블록1열 6,23,...,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1,32.0,1
4,1층 A블록1열 5,1층,A블록,1,5,1270,591,-93,1층 E블록1열 5,24,...,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1,32.0,1


(405810, 47)


# **예매된 경우 거래까지 걸린 시간 찾기**

In [3]:
def convert_string_to_hours(time_string):

    parts = time_string.split()
    time_parts = parts[2].split(':')
    
    days = int(parts[0])
    hour = int(time_parts[0])
    minutes = int(time_parts[1])

    return (days * 24) + hour + (minutes / 60)

def transaction_time(data):
    
    
    data = data.copy()
    data['거래까지걸린시간'] = np.NaN # default 값은 NaN 값을 맵핑하기 
    data['거래까지걸린시간_시간'] = np.NaN # default 값은 NaN 값을 맵핑하기 
    
    
    # 조건 찾기 
    purchase_cond = data['예매여부'] == 1 # 예매 된 경우에 대한 조건 
    open_cond = data['pre_open_date'].isna() # 선 예매가 되지 않은 경우에 대한 조건 
    
    # 조건에 따라 다르게 맵핑하기 
    
    open_time = pd.to_datetime(data.loc[(purchase_cond) & (open_cond), 'open_date'])
    open_purchase_time = pd.to_datetime(data.loc[(purchase_cond) & (open_cond), '전체거래시간'])
    
    pre_open_time = pd.to_datetime(data.loc[(purchase_cond) & (~open_cond), 'pre_open_date'])
    pre_open_purchase_time = pd.to_datetime(data.loc[(purchase_cond) & (~open_cond), '전체거래시간'])
    
    data.loc[(purchase_cond) & (open_cond),'거래까지걸린시간'] = open_purchase_time - open_time
    data.loc[(purchase_cond) & (~open_cond),'거래까지걸린시간'] = pre_open_purchase_time - pre_open_time
    
    data.loc[purchase_cond, '거래까지걸린시간_시간'] = data.loc[purchase_cond,'거래까지걸린시간'].map(lambda x: convert_string_to_hours(str(x)))
    
    return data
    
    

In [4]:
result = transaction_time(df)

In [5]:
display(result.head())
display(result.tail())
print(result.shape)

,seat,층,블록,열,넘버,X,Y,Z,대칭점,무대_밑_좌우시야각,...,공연월,공연일,공연연월,전체공연시간,전체거래시간,누적검색량,거래까지걸린시간,거래까지걸린시간_시간,거래까지걸린시간_등수,좌석 오픈 여부
0,1층 A블록1열 1,1층,A블록,1,1,1451,542,-93,1층 E블록1열 9,21,...,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1,32.0,1
1,1층 A블록1열 2,1층,A블록,1,2,1406,555,-93,1층 E블록1열 8,22,...,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1,32.0,1
2,1층 A블록1열 3,1층,A블록,1,3,1361,568,-93,1층 E블록1열 7,23,...,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1,32.0,1
3,1층 A블록1열 4,1층,A블록,1,4,1315,580,-93,1층 E블록1열 6,23,...,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1,32.0,1
4,1층 A블록1열 5,1층,A블록,1,5,1270,591,-93,1층 E블록1열 5,24,...,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1,32.0,1


,seat,층,블록,열,넘버,X,Y,Z,대칭점,무대_밑_좌우시야각,...,공연월,공연일,공연연월,전체공연시간,전체거래시간,누적검색량,거래까지걸린시간,거래까지걸린시간_시간,거래까지걸린시간_등수,좌석 오픈 여부
405805,합창석 H블록4열 8,합창석,H블록,4,8,1390,-121,450,합창석 F블록4열 5,19,...,6,2022-06-09,2022-06-01,2022-06-09 19:30:00,NaN,460,NaN,NaN,1.0,1
405806,합창석 H블록4열 9,합창석,H블록,4,9,1405,-74,450,합창석 F블록4열 4,19,...,6,2022-06-09,2022-06-01,2022-06-09 19:30:00,NaN,460,NaN,NaN,1.0,1
405807,합창석 H블록4열 10,합창석,H블록,4,10,1420,-28,450,합창석 F블록4열 3,20,...,6,2022-06-09,2022-06-01,2022-06-09 19:30:00,NaN,460,NaN,NaN,1.0,1
405808,합창석 H블록4열 11,합창석,H블록,4,11,1435,19,450,합창석 F블록4열 2,20,...,6,2022-06-09,2022-06-01,2022-06-09 19:30:00,NaN,460,NaN,NaN,1.0,1
405809,합창석 H블록4열 12,합창석,H블록,4,12,1451,65,450,합창석 F블록4열 1,20,...,6,2022-06-09,2022-06-01,2022-06-09 19:30:00,NaN,460,NaN,NaN,1.0,1


(405810, 47)


# **거래까지 걸린 시간을 구했으면 공연 별로 구매가 된 순서대로 맵핑해보자**

- 독립적인 공연 별로 구매가 가장 빨리 이뤄진 순서대로 오름차순으로 번호를 맵핑하고, 예매가 되지 않은 경우는 모두 2050 번째 좌석으로 맵핑해보자 
    
    - 만약 좌석이 오픈되지 않은 경우에는 어떻게 할까 ? 
    - 일단은 모두 오픈 여부와 상관없이 예매가 안된 경우는 최대 등수인 오픈된 좌석 수로 맵핑하도록 해보자



- 다만 고려해야 할 경우가 있다. 코로나 기간 때에는 좌석이 오픈되지 않은 경우들이 있기 때문에 좌석이 오픈되지 않은 경우에 대해서는 (옆 좌석에 이미 예매가 있을 경우 오픈이 되지 않았다.) 예매된 좌석과 유사한 등수를 가졌을 것이라 생각하고, 예매가 된 좌석들 중 k 개의 좌석의 등수로 평균화하여 생각하자

In [13]:
def during_transcation(data):
    
    covid_cond = (data['전체공연시간'] <= '2022-06-09 19:30:00') & (data['전체공연시간'] >= '2020-05-05 20:00:00') # 거리두기 기간
    
    
    # 거리두기 기간이 아닌 경우의 등수 
    
    not_covid_data = data[~covid_cond].copy()
    
    not_covid_unique_performance = sorted(not_covid_data['전체공연시간'].unique())
    
    not_covid_data['거래까지걸린시간_등수'] = np.NaN # default 값을 결측치로 설정, 거래가 이뤄지지 않은 좌석은 결측치로 남도록
        
    for uniq_perf in not_covid_unique_performance:
        
        cond = not_covid_data['전체공연시간'] == uniq_perf
        
        transaction_rank = sorted(not_covid_data.loc[cond,'거래까지걸린시간_시간'].unique())
        
        transaction_dict = {var : i  for i,var in enumerate(transaction_rank)} 
        
        not_covid_data.loc[cond,'거래까지걸린시간_등수'] = not_covid_data.loc[cond,'거래까지걸린시간_시간'].map(transaction_dict) # 거래까지 걸린 시간이 빠를 수록 1에 가깝게 맵핑
        
        not_covid_data.loc[not_covid_data['거래까지걸린시간_시간'].isna(), '거래까지걸린시간_등수'] = 2050 # 거래가 이뤄지지 않은 경우는 최대 오픈 좌석 개수인 2050 으로 맵핑 
                
                
    covid_data = data[covid_cond].copy() # 코로나 기간일 떄의 등수 측정하기 
    
    covid_unique_performance = sorted(covid_data['전체공연시간'].unique())
    
    covid_data['거래까지걸린시간_등수'] = np.NaN
    
    
    best_k_list = []
    k_mean_score = []
    k_mean_std = []
    open_num_list = []
    print('예매되지 않은 좌석의 등수 추정 중')
    
    for uniq_perf in tqdm(covid_unique_performance):
        
        perf_cond = covid_data['전체공연시간'] == uniq_perf  # 공연 시간에 대한 컨디션 
        open_cond = covid_data['좌석 오픈 여부'] == 1 # 좌석이 오픈 된 것에 대한 컨디션
        
        
        train_transaction_rank = sorted(covid_data.loc[(perf_cond) & (open_cond),'거래까지걸린시간_시간'].unique())
        train_transaction_dict = {var : i + 1 for i,var in enumerate(train_transaction_rank)} 
        
        covid_data.loc[perf_cond, '거래까지걸린시간_등수'] = covid_data.loc[perf_cond, '거래까지걸린시간_시간'].map(train_transaction_dict)

        train = covid_data.loc[(perf_cond) & (open_cond)] # 좌석이 오픈된 경우는 등수를 계산 할 수 있으며 학습에 사용 될 데이터 
        test = covid_data.loc[(perf_cond) & ~(open_cond)] # 좌석이 오픈되지 않은 경우는 등수를 추정해야 할 데이터
        
        # 좌석의 좌표와 등수만을 가지고 비오픈 지역을 추정하자 
        
        x_train,y_train = train[['X','Y','Z']] , train['거래까지걸린시간_등수'] 
        x_test,y_test = test[['X','Y','Z']] , test['거래까지걸린시간_등수']
        
        avg_scores_list = []
        std_list = []
        
        for k in range(2,9 + 1): # 최적의 k 개를 찾자 
            
            model = KNeighborsRegressor(n_neighbors = k, weights  = 'distance')
            scores = cross_val_score(model, x_train,y_train, cv = 30, scoring = 'neg_mean_squared_error')
            avg_scores_list.append(np.squeeze(np.mean(scores)))
            std_list.append(np.squeeze(np.std(scores)))
            
        best_k = np.argmin(avg_scores_list) + 2 # 최적의 k 개는 avg_scores_list 의 가장 낮을 수록 좋은 k 개 
        
        best_k_list.append(best_k)
        k_mean_score.append(np.abs(avg_scores_list[best_k - 2]))
        k_mean_std.append(std_list[best_k - 2])
        open_num_list.append(np.sum(covid_data.loc[perf_cond,'좌석 오픈 여부']))
        # best_k 를 추정했으면 학습해서 채워두자 
        
        best_model = KNeighborsRegressor(n_neighbors = best_k, weights = 'distance')
        best_model.fit(x_train,y_train)
        y_predict = best_model.predict(x_test).reshape(1,-1)
        y_predict += np.random.random(size = (1,len(y_predict))) # 예측값들의 등수가 동일하게 나오는 경우를 방지하기 위해 난수를 추가
        
        
        covid_data.loc[(perf_cond) & (~open_cond), '거래까지걸린시간_등수'] = np.squeeze(y_predict) # 해당 날짜와 추정된 조건의 거래까지 걸린 시간의 등수를 예측값으로 추정해두자 

        covid_data.loc[(perf_cond) & (open_cond) & (covid_data.loc[(perf_cond) & (open_cond),'거래까지걸린시간_시간'].isna()),'거래까지걸린시간_등수'] = np.sum(covid_data.loc[perf_cond, '좌석 오픈 여부']) # 최대 좌석 오픈 개수로 맵핑
        
                                                                                        
    knn_table = pd.DataFrame(
        {'전체공연시간': covid_unique_performance,
        'best_k' : best_k_list,
        'mse_score' : k_mean_score,
        'std' : k_mean_std,
        '좌석오픈개수' : open_num_list}
    )
    
    total_data = pd.concat([not_covid_data, covid_data])
    
    return total_data, knn_table

In [14]:
result , knn_table  = during_transcation(df)

예매되지 않은 좌석의 등수 추정 중


100%|██████████| 49/49 [00:39<00:00,  1.24it/s]


# **거리두기 기간동안 좌석이 오픈되지 않은 좌석의 KNN modeling table**

In [15]:
display(knn_table.sort_values(by = 'mse_score'))

,전체공연시간,best_k,mse_score,std,좌석오픈개수
7,2020-11-25 19:30:00,2,1.330610,2.145040,2259
37,2021-12-05 17:00:00,2,1.708125,2.466147,1776
44,2022-03-18 19:30:00,2,4.641455,6.010661,2129
3,2020-09-12 17:00:00,2,6.243525,6.918085,2330
32,2021-11-17 19:30:00,2,8.782986,7.168238,1678
24,2021-10-16 17:00:00,2,11.907571,9.953844,1427
6,2020-11-15 17:00:00,2,57.348052,66.623735,1688
0,2020-05-05 20:00:00,2,68.927282,69.805628,2156
34,2021-11-24 19:30:00,2,91.065397,97.934899,1758
35,2021-11-28 17:00:00,2,117.378687,125.861605,2345


In [16]:
# 예시의 경우를 통해 잘 되었는지 살펴보자 
# 코로나가 아닌 경우

result[result['전체공연시간'] == '2022-12-04 17:00:00'][['거래까지걸린시간_시간','거래까지걸린시간_등수']].sort_values(by = '거래까지걸린시간_등수').drop_duplicates() 

,거래까지걸린시간_시간,거래까지걸린시간_등수
225151,14.016667,1.0
225152,14.116667,2.0
223012,14.200000,3.0
223166,15.366667,4.0
225027,21.966667,5.0
...,...,...
223370,2152.833333,260.0
223984,2152.866667,261.0
223905,2152.983333,262.0
223345,2153.100000,263.0


In [9]:
# 거리두기 기간인 경우

result[result['전체공연시간'] == '2020-06-03 19:30:00'][['예매여부','좌석 오픈 여부','거래까지걸린시간_시간','거래까지걸린시간_등수']].sort_values(by = '거래까지걸린시간_등수').drop_duplicates() 

,예매여부,좌석 오픈 여부,거래까지걸린시간_시간,거래까지걸린시간_등수
287253,1,1,333.533333,1.00000
287557,1,1,343.683333,2.00000
287005,1,1,352.283333,3.00000
286985,1,1,561.400000,4.00000
286943,1,1,904.150000,5.00000
...,...,...,...,...
287185,0,0,NaN,121.62355
287184,1,1,4556.600000,122.00000
287109,1,1,4556.666667,123.00000
287113,1,1,4557.083333,124.00000


##### **잘 계산된 것으로 보인다 저장해주자**

In [10]:
file_path = 'C:/Users/whileduck/Desktop/빈좌석/'

result.to_csv(file_path + '클래식_빈좌석포함데이터.csv',index = False)  